# REDES NEURAIS ARTIFICIAIS
## Evaluate Improve e Tunning

# Parte 1 - Pré-processamento dos Dados

### Importar as libs

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

2021-08-20 18:50:41.897565: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'libcudart.so.10.1'; dlerror: libcudart.so.10.1: cannot open shared object file: No such file or directory
2021-08-20 18:50:41.897590: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
tf.__version__

'2.3.1'

### Importar o dataset

In [3]:
dataset = pd.read_csv('Churn_Modelling.csv')
dataset.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [4]:
X = dataset.iloc[:, 3:13].values
y = dataset.iloc[:, 13].values
print("X >>\n",X)
print("y >>\n",y)

X >>
 [[619 'France' 'Female' ... 1 1 101348.88]
 [608 'Spain' 'Female' ... 0 1 112542.58]
 [502 'France' 'Female' ... 1 0 113931.57]
 ...
 [709 'France' 'Female' ... 0 1 42085.58]
 [772 'Germany' 'Male' ... 1 0 92888.52]
 [792 'France' 'Female' ... 1 0 38190.78]]
y >>
 [1 0 1 ... 1 1 0]


### Transformando os dados categóricos


In [5]:
# Label Encoding the "Gender" column
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X[:, 2] = le.fit_transform(X[:, 2])
print(X)
# One Hot Encoding the "Geography" column
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough')
X = np.array(ct.fit_transform(X))
print(X)

[[619 'France' 0 ... 1 1 101348.88]
 [608 'Spain' 0 ... 0 1 112542.58]
 [502 'France' 0 ... 1 0 113931.57]
 ...
 [709 'France' 0 ... 0 1 42085.58]
 [772 'Germany' 1 ... 1 0 92888.52]
 [792 'France' 0 ... 1 0 38190.78]]
[[1.0 0.0 0.0 ... 1 1 101348.88]
 [0.0 0.0 1.0 ... 0 1 112542.58]
 [1.0 0.0 0.0 ... 1 0 113931.57]
 ...
 [1.0 0.0 0.0 ... 0 1 42085.58]
 [0.0 1.0 0.0 ... 1 0 92888.52]
 [1.0 0.0 0.0 ... 1 0 38190.78]]


# Dividindo o dataset em conjunto de treinamento e conjunto de teste

In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

# Feature Scaling

In [7]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [8]:
np.shape(X_train)

(8000, 12)

---
# Parte 2 -Vamos construir uma ANN!


### Initializing the ANN

In [9]:
ann = tf.keras.models.Sequential()

2021-08-20 18:50:45.189112: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2021-08-20 18:50:45.189145: W tensorflow/stream_executor/cuda/cuda_driver.cc:312] failed call to cuInit: UNKNOWN ERROR (303)
2021-08-20 18:50:45.189179: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (iurisevero-Inspiron-3583): /proc/driver/nvidia/version does not exist
2021-08-20 18:50:45.190370: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN)to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-08-20 18:50:45.216614: I tensorflow/core/platform/profile_utils/cpu_utils.cc:104] CPU Frequenc

### Adding the input layer and the first hidden layer

In [10]:
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))

### Adding the second hidden layer

In [11]:
ann.add(tf.keras.layers.Dense(units=6, activation='tanh'))

### Adding the output layer

In [12]:
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

## Part 3 - Training the ANN

### Compiling the ANN

In [13]:
ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

### Training the ANN on the Training set

In [14]:
ann.fit(X_train, y_train, batch_size = 32, epochs = 100)

Epoch 1/100
250/250 [==============================] - 0s 650us/step - loss: 0.5398 - accuracy: 0.7636
Epoch 2/100
250/250 [==============================] - 0s 707us/step - loss: 0.4671 - accuracy: 0.7961
Epoch 3/100
250/250 [==============================] - 0s 650us/step - loss: 0.4424 - accuracy: 0.7987
Epoch 4/100
250/250 [==============================] - 0s 703us/step - loss: 0.4201 - accuracy: 0.8148
Epoch 5/100
250/250 [==============================] - 0s 738us/step - loss: 0.4016 - accuracy: 0.8285
Epoch 6/100
250/250 [==============================] - 0s 658us/step - loss: 0.3883 - accuracy: 0.8375
Epoch 7/100
250/250 [==============================] - 0s 662us/step - loss: 0.3786 - accuracy: 0.8461
Epoch 8/100
250/250 [==============================] - 0s 674us/step - loss: 0.3714 - accuracy: 0.8508
Epoch 9/100
250/250 [==============================] - 0s 695us/step - loss: 0.3654 - accuracy: 0.8535
Epoch 10/100
250/250 [==============================] - 0s 719us/step - l

# Parte 3 - Fazendo predições e avaliando o modelo

## Prevendo os resultados com o conjunto de testes

In [15]:
y_pred = ann.predict(X_test)
y_pred

array([[0.26571813],
       [0.2788602 ],
       [0.16619414],
       ...,
       [0.24224257],
       [0.14167663],
       [0.1815126 ]], dtype=float32)

In [16]:
y_pred = (y_pred > 0.5)
y_pred

array([[False],
       [False],
       [False],
       ...,
       [False],
       [False],
       [False]])

# Criando uma Confusion Matrix

In [17]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)

[[1515   80]
 [ 195  210]]


---
# Avaliando o Modelo

In [18]:
# Evaluating the ANN
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [19]:
def build_classifier():
    classifier = Sequential()
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 12))
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'tanh'))
    classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
    classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier

In [20]:
classifier = KerasClassifier(build_fn = build_classifier, batch_size = 10, epochs = 10)
accuracies = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10, n_jobs = -1)
mean = accuracies.mean()
variance = accuracies.std()

2021-08-20 18:51:06.184413: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'libcudart.so.10.1'; dlerror: libcudart.so.10.1: cannot open shared object file: No such file or directory
2021-08-20 18:51:06.184449: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2021-08-20 18:51:06.198738: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'libcudart.so.10.1'; dlerror: libcudart.so.10.1: cannot open shared object file: No such file or directory
2021-08-20 18:51:06.198763: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2021-08-20 18:51:06.231099: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'libcudart.so.10.1'; dlerror: libcudart.so.10.1: cannot open shared object file: No such file or dire

Epoch 1/10
720/720 [==============================] - 1s 2ms/step - loss: 0.4910 - accuracy: 0.7935
Epoch 2/10
720/720 [==============================] - 1s 2ms/step - loss: 0.4240 - accuracy: 0.8032
Epoch 3/10
720/720 [==============================] - 1s 2ms/step - loss: 0.4235 - accuracy: 0.7992
Epoch 3/10
720/720 [==============================] - 1s 2ms/step - loss: 0.4257 - accuracy: 0.7969
Epoch 3/10
720/720 [==============================] - 1s 2ms/step - loss: 0.4209 - accuracy: 0.8219
Epoch 4/10
720/720 [==============================] - 1s 2ms/step - loss: 0.4163 - accuracy: 0.8292
Epoch 4/10
720/720 [==============================] - 1s 2ms/step - loss: 0.4173 - accuracy: 0.8268
Epoch 4/10
720/720 [==============================] - 2s 2ms/step - loss: 0.4134 - accuracy: 0.8272
Epoch 4/10
720/720 [==============================] - 2s 2ms/step - loss: 0.4151 - accuracy: 0.8313
Epoch 4/10
720/720 [==============================] - 2s 2ms/step - loss: 0.4079 - accuracy: 0.8346


In [21]:
print(mean,variance)

0.8326249957084656 0.009975762689936636


# Melhorando o modelo

In [30]:
# Improving the ANN
# Dropout Regularization to reduce overfitting if needed
from tensorflow.keras.layers import Dropout

def build_classifier():
    classifier = Sequential()
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 12))
    classifier.add(Dropout(rate = 0.1))
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'tanh'))
    classifier.add(Dropout(rate = 0.1))
    classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
    classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier

In [31]:
classifier = KerasClassifier(build_fn = build_classifier, batch_size = 10, epochs = 10)
accuracies = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10, n_jobs = -1)
mean = accuracies.mean()
variance = accuracies.std()

2021-08-20 19:03:11.587498: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'libcudart.so.10.1'; dlerror: libcudart.so.10.1: cannot open shared object file: No such file or directory
2021-08-20 19:03:11.588100: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2021-08-20 19:03:11.598218: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'libcudart.so.10.1'; dlerror: libcudart.so.10.1: cannot open shared object file: No such file or directory
2021-08-20 19:03:11.598343: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2021-08-20 19:03:11.651807: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'libcudart.so.10.1'; dlerror: libcudart.so.10.1: cannot open shared object file: No such file or dire

Epoch 1/10
720/720 [==============================] - 2s 2ms/step - loss: 0.4965 - accuracy: 0.7951
Epoch 2/10
720/720 [==============================] - 2s 2ms/step - loss: 0.4987 - accuracy: 0.7961
Epoch 2/10
720/720 [==============================] - 2s 2ms/step - loss: 0.4943 - accuracy: 0.7951
Epoch 2/10
720/720 [==============================] - 2s 2ms/step - loss: 0.4978 - accuracy: 0.7936
Epoch 2/10
720/720 [==============================] - 2s 2ms/step - loss: 0.4954 - accuracy: 0.7956
Epoch 2/10
720/720 [==============================] - 2s 2ms/step - loss: 0.4368 - accuracy: 0.7969
Epoch 3/10
720/720 [==============================] - 2s 2ms/step - loss: 0.4333 - accuracy: 0.7944
Epoch 3/10
720/720 [==============================] - 2s 2ms/step - loss: 0.4327 - accuracy: 0.7962
Epoch 3/10
720/720 [==============================] - 2s 2ms/step - loss: 0.4276 - accuracy: 0.7976
Epoch 3/10
720/720 [==============================] - 2s 3ms/step - loss: 0.4265 - accuracy: 0.8018


In [41]:
print(mean,variance)

0.8314999997615814 0.012522487121245755


# Tuning the ANN

In [32]:
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

def build_classifier(optimizer):
    classifier = Sequential()
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 12))
    classifier.add(Dropout(rate = 0.1))
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'tanh'))
    classifier.add(Dropout(rate = 0.1))
    classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
    classifier.compile(optimizer = optimizer, loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier

classifier = KerasClassifier(build_fn = build_classifier)

parameters = {
    'batch_size': [10, 25, 50],
    'epochs': [50, 100],
    'optimizer': ['adam', 'rmsprop']
}

grid_search = GridSearchCV(estimator = classifier,
                           scoring = 'accuracy',
                           param_grid = parameters,
                           cv = 10)

In [ ]:
%%time

grid_search = grid_search.fit(X_train, y_train)

Epoch 1/50
720/720 [==============================] - 0s 462us/step - loss: 0.4984 - accuracy: 0.7961
Epoch 2/50
720/720 [==============================] - 0s 594us/step - loss: 0.4328 - accuracy: 0.7971
Epoch 3/50
720/720 [==============================] - 0s 611us/step - loss: 0.4258 - accuracy: 0.8067
Epoch 4/50
720/720 [==============================] - 0s 666us/step - loss: 0.4198 - accuracy: 0.8292
Epoch 5/50
720/720 [==============================] - 1s 697us/step - loss: 0.4184 - accuracy: 0.8310
Epoch 6/50
720/720 [==============================] - 1s 695us/step - loss: 0.4152 - accuracy: 0.8285
Epoch 7/50
720/720 [==============================] - 0s 688us/step - loss: 0.4165 - accuracy: 0.8293
Epoch 8/50
720/720 [==============================] - 0s 655us/step - loss: 0.4141 - accuracy: 0.8276
Epoch 9/50
720/720 [==============================] - 1s 774us/step - loss: 0.4141 - accuracy: 0.8324
Epoch 10/50
720/720 [==============================] - 0s 555us/step - loss: 0.416

In [40]:
print('Best params:\n',grid_search.best_params_,'\nBest accuracy:\n',grid_search.best_score_)

Best params:
 {'batch_size': 10, 'epochs': 100, 'optimizer': 'rmsprop'} 
Best accuracy:
 0.8375


# Halving Grid Search

In [ ]:
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingGridSearchCV

def build_classifier(optimizer):
    classifier = Sequential()
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 12))
    classifier.add(Dropout(rate = 0.1))
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'tanh'))
    classifier.add(Dropout(rate = 0.1))
    classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
    classifier.compile(optimizer = optimizer, loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier

classifier = KerasClassifier(build_fn = build_classifier)

parameters = {
    'batch_size': [10, 25, 50],
    'epochs': [50, 100],
    'optimizer': ['adam', 'rmsprop']
}

halving_cv = HalvingGridSearchCV(estimator = classifier,
                                  param_grid = parameters,
                                  scoring="accuracy", 
                                  n_jobs=-1, 
                                  min_resources="exhaust", 
                                  factor=3,
                                  cv = 10)

In [ ]:
%%time
halving_cv = halving_cv.fit(X_train, y_train)

2021-08-20 20:01:16.380408: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'libcudart.so.10.1'; dlerror: libcudart.so.10.1: cannot open shared object file: No such file or directory
2021-08-20 20:01:16.380408: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'libcudart.so.10.1'; dlerror: libcudart.so.10.1: cannot open shared object file: No such file or directory
2021-08-20 20:01:16.380430: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2021-08-20 20:01:16.380435: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2021-08-20 20:01:16.385403: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'libcudart.so.10.1'; dlerror: libcudart.so.10.1: cannot open shared object file: No such file or dire

Epoch 1/50
80/80 [==============================] - 0s 1ms/step - loss: 0.6753 - accuracy: 0.7910
Epoch 2/50
80/80 [==============================] - 0s 1ms/step - loss: 0.6710 - accuracy: 0.8098
Epoch 2/50
80/80 [==============================] - 0s 2ms/step - loss: 0.5895 - accuracy: 0.7960
Epoch 2/50
80/80 [==============================] - 0s 2ms/step - loss: 0.6744 - accuracy: 0.7835
Epoch 2/50
80/80 [==============================] - 0s 2ms/step - loss: 0.6755 - accuracy: 0.7722
Epoch 2/50
80/80 [==============================] - 0s 3ms/step - loss: 0.6774 - accuracy: 0.7635
Epoch 2/50
80/80 [==============================] - 0s 3ms/step - loss: 0.5935 - accuracy: 0.7922
Epoch 3/50
80/80 [==============================] - 0s 3ms/step - loss: 0.5897 - accuracy: 0.7847
Epoch 3/50
80/80 [==============================] - 0s 2ms/step - loss: 0.5804 - accuracy: 0.8098
Epoch 3/50
80/80 [==============================] - 0s 3ms/step - loss: 0.4652 - accuracy: 0.8098
Epoch 4/50
80/80 [==

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


80/80 [==============================] - 0s 2ms/step - loss: 0.3735 - accuracy: 0.8436
Epoch 49/50
80/80 [==============================] - 0s 2ms/step - loss: 0.3746 - accuracy: 0.8385
Epoch 46/50
80/80 [==============================] - 0s 2ms/step - loss: 0.4118 - accuracy: 0.8173
Epoch 47/50
80/80 [==============================] - 0s 2ms/step - loss: 0.3867 - accuracy: 0.8385
Epoch 49/50
80/80 [==============================] - 0s 2ms/step - loss: 0.3801 - accuracy: 0.8473


Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


...

Epoch 48/50
80/80 [==============================] - 0s 2ms/step - loss: 0.4091 - accuracy: 0.8185
Epoch 49/50
80/80 [==============================] - 0s 2ms/step - loss: 0.3869 - accuracy: 0.8436
Epoch 49/50
80/80 [==============================] - 0s 2ms/step - loss: 0.3870 - accuracy: 0.8360
Epoch 49/50
80/80 [==============================] - 0s 2ms/step - loss: 0.4094 - accuracy: 0.8198
Epoch 50/50
80/80 [==============================] - 0s 2ms/step - loss: 0.3999 - accuracy: 0.8360
Epoch 50/50
80/80 [==============================] - 0s 2ms/step - loss: 0.4154 - accuracy: 0.8135


Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation)

80/80 [==============================] - 0s 2ms/step - loss: 0.4097 - accuracy: 0.8298
Epoch 50/50
80/80 [==============================] - 0s 2ms/step - loss: 0.4124 - accuracy: 0.8335


Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


Epoch 1/50
80/80 [==============================] - 0s 2ms/step - loss: 0.6743 - accuracy: 0.7997
Epoch 2/50
80/80 [==============================] - 0s 2ms/step - loss: 0.5750 - accuracy: 0.8085
Epoch 3/50
80/80 [==============================] - 0s 2ms/step - loss: 0.6750 - accuracy: 0.7860
Epoch 2/50
80/80 [==============================] - 0s 2ms/step - loss: 0.4745 - accuracy: 0.8085
Epoch 4/50
80/80 [==============================] - 0s 2ms/step - loss: 0.5903 - accuracy: 0.7897
Epoch 3/50
80/80 [==============================] - 0s 2ms/step - loss: 0.4460 - accuracy: 0.8085
Epoch 4/50
80/80 [==============================] - 0s 2ms/step - loss: 0.6721 - accuracy: 0.7672
Epoch 2/50
80/80 [==============================] - 0s 2ms/step - loss: 0.6700 - accuracy: 0.7710
Epoch 2/50
80/80 [==============================] - 0s 2ms/step - loss: 0.4343 - accuracy: 0.8085
Epoch 6/50
80/80 [==============================] - 0s 2ms/step - loss: 0.6082 - accuracy: 0.7747
Epoch 3/50
80/80 [==

In [43]:
print('Best params:\n',halving_cv.best_params_,'\nBest accuracy:\n',halving_cv.best_score_)

Best params:
 {'batch_size': 10, 'epochs': 100, 'optimizer': 'adam'} 
Best accuracy:
 0.8355444305381727


In [46]:
from tensorflow.keras.layers import Dropout

def build_classifier():
    classifier = Sequential()
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 12))
    classifier.add(Dropout(rate = 0.1))
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'tanh'))
    classifier.add(Dropout(rate = 0.1))
    classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
    classifier.compile(optimizer = 'rmsprop', loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier

classifier = KerasClassifier(build_fn = build_classifier, batch_size = 10, epochs = 100)
accuracies = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10, n_jobs = -1)
mean = accuracies.mean()
variance = accuracies.std()
print(mean,variance)

Epoch 1/100
720/720 [==============================] - 1s 1ms/step - loss: 0.5060 - accuracy: 0.7943
Epoch 2/100
720/720 [==============================] - 1s 1ms/step - loss: 0.5036 - accuracy: 0.7975
Epoch 2/100
720/720 [==============================] - 1s 1ms/step - loss: 0.5106 - accuracy: 0.7961
Epoch 2/100
720/720 [==============================] - 1s 1ms/step - loss: 0.5076 - accuracy: 0.7926
Epoch 2/100
720/720 [==============================] - 1s 1ms/step - loss: 0.5113 - accuracy: 0.7950
Epoch 2/100
720/720 [==============================] - 1s 1ms/step - loss: 0.5019 - accuracy: 0.7965
Epoch 2/100
720/720 [==============================] - 1s 1ms/step - loss: 0.4373 - accuracy: 0.7944
Epoch 3/100
720/720 [==============================] - 1s 1ms/step - loss: 0.4405 - accuracy: 0.7969
Epoch 3/100
720/720 [==============================] - 1s 1ms/step - loss: 0.4343 - accuracy: 0.7975
Epoch 3/100
720/720 [==============================] - 1s 1ms/step - loss: 0.4344 - accurac

# FIM

> Professor Diego Dorgam  
> [@diegodorgam](https://twitter.com/diegodorgam)